## Crawler for Yelp Reviews

In [1]:
import time
import pandas as pd
import os

# packages to scrap content from web pages
import selenium
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# interpretation of html 
from bs4 import BeautifulSoup

#### Constants

In [2]:
URL_BODY = "https://www.yelp.com/search?find_desc=$1&find_loc=$2&start=$3"
RESULT_PATH = r"./"
RESULT_FILE = r"yelp_restaurants.csv"

#### entities to crawl

In [3]:
l_search_entities = ["Chick-fil-A", "Chipotle Mexican Grill", "Burger King", "KFC", "McDonald’s", "Subway Restaurants", "Taco Bell",
"Wendy’s", "Dunkin’", "Papa John’s Pizza", "Pizza+Hut", "Starbucks", "Applebee’s+Grill+%2B+Bar", "Olive Garden Italian Restaurant"
]

l_search_entities = [
"Dunkin’", "Papa John’s Pizza", "Pizza+Hut", "Starbucks", "Applebee’s+Grill+%2B+Bar", "Olive Garden Italian Restaurant"
]

d_places = {'New York': 'New+York%2C+NY%2C+Vereinigte+Staaten', 'Los Angeles': 'Los Angeles%2C CA', 'Chicago': 'Chicago%2C IL', 'Houston': 'Houston%2C TX', 'Phoenix': 'Phoenix%2C AZ', 'Philadelphia': 'Philadelphia%2C PA', 'San Antonio': 'San Antonio%2C TX', 'San Diego': 'San Diego%2C CA', 'Dallas': 'Dallas%2C TX', 'San Jose': 'San Jose%2C CA', 'Austin': 'Austin%2C TX', 'Jacksonville': 'Jacksonville%2C FL', 'Fort Worth': 'Fort Worth%2C TX', 'Columbus': 'Columbus%2C OH', 'San Francisco': 'San Francisco%2C CA',
'Charlotte': 'Charlotte%2C NC', 'Indianapolis': 'Indianapolis%2C IN', 'Seattle': 'Seattle%2C WA', 'Denver': 'Denver%2C CO', 'Washington': 'Washington%2C DC', 'Boston': 'Boston%2C MA', 'El Paso': 'El Paso%2C TX', 'Detroit': 'Detroit%2C MI', 'Nashville': 'Nashville%2C TN', 'Portland': 'Portland%2C OR', 'Memphis': 'Memphis%2C TN', 'Oklahoma City': 'Oklahoma City%2C OK', 
'Las Vegas': 'Las Vegas%2C NV', 'Louisville': 'Louisville%2C KY', 'Baltimore': 'Baltimore%2C MD', 'Milwaukee': 'Milwaukee%2C WI', 'Albuquerque': 'Albuquerque%2C NM', 'Tucson': 'Tucson%2C AZ', 'Fresno': 'Fresno%2C CA', 'Mesa': 'Mesa%2C AZ', 
'Sacramento': 'Sacramento%2C CA', 'Atlanta': 'Atlanta%2C GA', 'Kansas City': 'Kansas City%2C MO', 'Colorado Springs': 'Colorado Springs%2C CO', 'Miami': 'Miami%2C FL', 'Raleigh': 'Raleigh%2C NC', 'Omaha': 'Omaha%2C NE', 
'Long Beach': 'Long Beach%2C CA', 'Virginia Beach': 'Virginia Beach%2C VA', 'Oakland': 'Oakland%2C CA', 'Minneapolis': 'Minneapolis%2C MN', 'Tulsa': 'Tulsa%2C OK', 'Tampa': 'Tampa%2C FL', 'New Orleans': 'New Orleans%2C LA'
}

#### init driver 

In [4]:
#driver = webdriver.Chrome(r"P:\chromedriver_win32\chromedriver.exe")
driver = webdriver.Chrome(r"C:/chromedriver.exe")

driver.implicitly_wait(5)


#### Init lists of result data

In [5]:
l_link = []
l_restaurant = []
l_place = []
l_rating = []
l_date = []
l_text = []

#### Function to crawl links from restaurants from overview page
**input:** name of restaurant like 'Subway' (**string**), location like 'Boston' (**string**) <br>
**output:** list of links to all found items

In [18]:
def get_items(item_name, place):
    l_links = []
    
    while True:
        driver.get(URL_BODY.replace("$1", item_name).replace("$2", place).replace("$3", str(len(l_links))))
        time.sleep(0.6)
        soup = BeautifulSoup(driver.page_source, features="html.parser")

        items = soup.select('a[class="css-1422juy"]')

        items = list(filter(lambda x: x.has_attr('name') and x['name'] == item_name, items))
        
        if len(items) == 0: break
        
        l_links += list(map(lambda x : x['href'], items))
        
    
    
    return l_links
    

#### Testing (not mandatory to execute)

In [7]:
restaurants = get_items("McDonald’s", 'Los Angeles%2C CA')
print(restaurants)

['/biz/mcdonalds-los-angeles-114?osq=McDonald%27s', '/biz/mcdonalds-los-angeles-106?osq=McDonald%27s', '/biz/mcdonalds-los-angeles-38?osq=McDonald%27s', '/biz/mcdonalds-los-angeles-43?osq=McDonald%27s', '/biz/mcdonalds-los-angeles-43?osq=McDonald%27s']


#### Function to crawl reviews to a single item
**input:** link to a certain restaurant/ item (**string**) (provided by function 'get_items()' above) <br>
**output:** list of lists of rating, comment and date

In [19]:
from tqdm import tqdm
from selenium.common.exceptions import StaleElementReferenceException, ElementClickInterceptedException


def handle_reviews(link):
    driver.get("https://www.yelp.com%s&sort_by=date_desc" % link)
    
    time.sleep(1)

    l_ratings = []
    l_comments = []
    l_dates = []

    page = 1
    while True:
        time.sleep(0.8)
        soup = BeautifulSoup(driver.page_source, features="html.parser")
        section_ratings = soup.select('section[class="margin-t4__09f24__G0VVf padding-t4__09f24__Y6aGL border--top__09f24__exYYb border-color--default__09f24__NPAKY"]')

        if not section_ratings:
            print('no selection_ratings')
            return [l_ratings, l_dates, l_comments]

        # print(f'Comment page {page}, length of section ratings: {len(section_ratings)}')
        for i in range(len(section_ratings)):
            soup_ratings = BeautifulSoup(str(section_ratings[i]), features="html.parser")
            if not soup_ratings:
                continue

            ratings = soup_ratings.select('div[class="review__09f24__oHr9V border-color--default__09f24__NPAKY"]')
            if not ratings:
                continue
            # print(f'length of ratings: {len(ratings)}')
            for j in range(len(ratings)):
                # print(ratings[i])
                soup_detail = BeautifulSoup(str(ratings[j]), features="html.parser")

                if soup_detail is None:
                    continue

                rating = soup_detail.select_one('div[class*="i-stars__09f24__foihJ"]')
                rating = rating['aria-label'] if rating else ''

                comment = soup_detail.select_one('span[class*="raw__09f24__T4Ezm"]')
                comment = comment.text if comment else ''

                date = soup_detail.select_one('span[class*="css-1e4fdj9"]')
                date = date.text if date else ''

                l_ratings.append(str(rating))
                l_comments.append(str(comment))
                l_dates.append(str(date))

        try:
            next_comment_page = driver.find_element_by_css_selector('a[class*="next-link navigation-button__09f24__m9qRz"]')
        except NoSuchElementException:
            break

        next_comment_page.click()
        page += 1
        
    
    return [l_ratings, l_dates, l_comments]

In [10]:
test_review = handle_reviews(restaurants[1])
print(test_review[2][-2])

#### Function to read already handled links

In [20]:
def get_list_of_handled_links():
    if not os.path.exists(RESULT_PATH + RESULT_FILE):
        # df_temp = pd.DataFrame()
        # df_temp.to_csv(columns=['link', 'restaurant', 'place', 'rating', 'date', 'comment'])
        return None
    df = pd.read_csv(RESULT_FILE, sep=";", header=0, encoding = "ISO-8859-1")
    if "link" in df:
        return df["link"].tolist()
    else:
        return None

#### Main 

In [ ]:

handled_links = get_list_of_handled_links()
print(handled_links)
for item in l_search_entities:
    
    for place in d_places:
        # first, get all restaurant links
        links = get_items(item, d_places[place])

        if handled_links:
            links_filtered = list(filter(lambda x: x not in handled_links, links))
        else:
            links_filtered = list(links)
            
        for link in links_filtered:
            try:
                # then, get (rating, comments, date)
                l_results = handle_reviews(link)
            except IndexError:
                continue
            
            l_link += [link] * len(l_results[0])
            l_restaurant += [item] * len(l_results[0])
            l_place += [place] * len(l_results[0])
            l_rating += l_results[0]
            l_date += l_results[1]
            l_text += l_results[2]
                

        print("%s - %s, %d restaurants (%d already handled)" % (item, place, len(links), len(links) - len(links_filtered)))
        print("")
    

None


#### Building DataFrame of crawled data

In [15]:
df_grid = pd.DataFrame()


df_grid["link"] = l_link
df_grid["restaurant"] = l_restaurant
df_grid["place"] = l_place
df_grid["rating"] = l_rating
df_grid["date"] = l_date
df_grid["comment"] = l_text

df_grid.to_csv(RESULT_PATH + RESULT_FILE)


#### Visualize results

In [11]:
df_grid

,link,restaurant,place,rating,date,comment
0,/biz/dunkin-boston-43?osq=Dunkin%27,Dunkin’,Boston,5 Sterne,24.9.2015,First off let me just say that I've never writ...
1,/biz/dunkin-boston-43?osq=Dunkin%27,Dunkin’,Boston,3 Sterne,29.3.2018,This Dunkin' happens to be convenient on my ro...
2,/biz/dunkin-boston-43?osq=Dunkin%27,Dunkin’,Boston,2 Sterne,17.8.2018,"Visited 8/17/18 @ 1:06 pmCoffee, iced tea, and..."
3,/biz/dunkin-boston-43?osq=Dunkin%27,Dunkin’,Boston,3 Sterne,20.7.2018,The employees are always friendly and they wor...
4,/biz/dunkin-boston-43?osq=Dunkin%27,Dunkin’,Boston,4 Sterne,9.7.2017,"Staff is nice, but it's just a tough area. A l..."
5,/biz/dunkin-boston-43?osq=Dunkin%27,Dunkin’,Boston,4 Sterne,14.4.2014,Got the Butter Pecan iced coffee and it was AM...
6,/biz/dunkin-boston-43?osq=Dunkin%27,Dunkin’,Boston,4 Sterne,26.6.2015,The folks that work at this dunks are some tou...
7,/biz/dunkin-boston-43?osq=Dunkin%27,Dunkin’,Boston,5 Sterne,4.11.2013,Huge store with parking lot right next to Andr...
8,/biz/dunkin-boston-43?osq=Dunkin%27,Dunkin’,Boston,3 Sterne,5.1.2016,Convenient and quick which is all I can ask fo...
9,/biz/dunkin-boston-43?osq=Dunkin%27,Dunkin’,Boston,4 Sterne,10.10.2010,"Like any other Dunkin, but this is one right o..."


#### Export data

In [13]:
if os.path.exists(RESULT_FILE):

    df_existing_data = pd.read_csv(RESULT_PATH, sep=";", decimal=",", header=0)
    df_existing_data = df_existing_data.append(df_grid)
    df_existing_data.drop_duplicates(inplace = True)
    df_existing_data.to_csv(RESULT_FILE, index = False, sep=";", decimal=",")

else:
    
    df_grid.to_csv(RESULT_FILE, index = False, sep=";", decimal=",")